In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

In [2]:
from graphviz import Digraph

In [3]:
import hiddenlayer as hl

In [8]:
import os
import sys
from os.path import abspath, expanduser
sys.path.append(abspath(expanduser("/../..")))
sys.path.append(abspath(expanduser("..")))
from sequence_annotation.processor.worker.worker import Worker
from IndRNN_pytorch.cuda_IndRNN_onlyrecurrent import IndRNN_onlyrecurrent

ImportError: No module named 'cupy'

In [5]:
"""This submodule provides trainer to train model"""

class TrainWorker(Worker):
    """a worker which will train and evaluate the model"""
    def __init__(self,path_root=None,is_verbose_visible=True):
        super().__init__(path_root)
        self._result = {}
        self.is_verbose_visible = is_verbose_visible

    def before_work(self):
        if self._path_root is not None:
            root_path = './'+self._path_root
            create_folder(root_path+"/model")
            create_folder(root_path+"/log")
            create_folder(root_path+"/result")
            plot_saved_path = root_path+"/model_image.png"
            #plot_model(self.model,show_shapes=True,to_file=plot_saved_path)
            length = str(len(str(self._epoch)))
            model_saved_path = root_path+'/model/epoch_{:0'+length+'d}.h5'
            model_saved_path = model_saved_path.format(0)
            #self.model.save(model_saved_path)

    def work(self):
        """Train model"""
        self._validate()
        self.wrapper(self.model,self.data,self.compiler.optimizer,self.compiler.loss)

    def after_work(self):
        """Do something after worker work"""
        pass

In [6]:
import deepdish as dd
from sequence_annotation.data_handler.fasta import read_fasta
from sequence_annotation.genome_handler.sequence import AnnSequence
from sequence_annotation.genome_handler.seq_container import AnnSeqContainer
from sequence_annotation.pipeline.data_generator import SeqGenerator
import random
h5=dd.io.load('../io/data/2018_11_24/Araport11_exon_2018_11_24_merged_with_coordinate_file_megred_exon_dist_to_five_1000_dist_to_three_500_merged.h5')
fasta = read_fasta('../io/data/2018_11_24/Araport11_exon_2018_11_24_merged_with_coordinate_file_megred_exon_dist_to_five_1000_dist_to_three_500_merged.fasta')
ann_seqs = AnnSeqContainer().from_dict(h5)
from numpy import median
median_len = median([len(seq) for seq in ann_seqs])
median_fasta = {}
for seq in ann_seqs:
    if len(seq)<= median_len:
        median_fasta[seq.id]=fasta[seq.id]
keys = list(median_fasta.keys())
random.shuffle(keys)
selected_seqs = AnnSeqContainer()
selected_seqs.ANN_TYPES = ann_seqs.ANN_TYPES
selected_fasta = {}
number = 0
for seq_id in keys:
    selected_seqs.add(ann_seqs.get(seq_id))
    selected_fasta[seq_id]=median_fasta[seq_id]
    number += 1
    if number >= int(10):
        break

Using TensorFlow backend.


In [7]:
from torch.autograd import Variable 
class CategoricalMetric:
    def __init__(self, class_num, ignore_index=None):
        self._class_num = class_num
        self._ignore_index = ignore_index
        self._metric = {}
        self.reset()
    @property
    def metric(self):
        return self._metric
    @property
    def accuracy(self):
        T = self._metric['T']
        F = self._metric['F']
        return T/(T+F)
    def reset(self):
        for type_ in ['TP','FP','TN','FN']:
            for index in range(self._class_num):
                self._metric[type_+"_"+str(index)] = 0
        self._metric['T'] = 0
        self._metric['F'] = 0
    def __call__(self,outputs,labels):
        if self._ignore_index is not None:
            mask = labels != self._ignore_index
        T_ = (outputs == labels)
        F_ = (outputs != labels)
        if self._ignore_index is not None:
            T_ = T_*mask
            F_ = F_*mask
        self._metric['T'] += T_.sum().item()
        self._metric['F'] += F_.sum().item()

        for index in range(self._class_num):
            P = outputs == index
            R = labels == index
            TP_ = P & R
            TN_ = ~P & ~R
            FP_ = P & ~R
            FN_ = ~P & R
            if self._ignore_index is not None:
                TP_ = (TP_*mask)
                TN_ = (TN_*mask)
                FP_ = (FP_*mask)
                FN_ = (FN_*mask)
            TP = TP_.sum().item()
            TN = TN_.sum().item()
            FP = FP_.sum().item()
            FN = FN_.sum().item()
            self._metric["TP_"+str(index)] += TP
            self._metric["FP_"+str(index)] += FP
            self._metric["TN_"+str(index)] += TN
            self._metric["FN_"+str(index)] += FN
        


In [8]:
from sequence_annotation.processor.compiler.compiler import Compiler
from sequence_annotation.processor.metric.metric_builder import MetricBuilder
import torch.optim as optim

class SimpleCompiler(Compiler):
    def __init__(self,optimizer,loss_type,metrics=None):
        self._optimizer_wrapper = optimizer

        super().__init__(None,loss_type,metrics)
    @property
    def loss(self):
        return self._loss_type
    @property
    def optimizer(self):
        return self._optimizer
    def process(self,model):
        self._optimizer = self._optimizer_wrapper(model)
        
    def before_process(self,path=None):
        if path is not None:
            json_path = create_folder(path) + "/setting/compiler.json"
            with open(json_path,'w') as fp:
                json.dump(self._record,fp)


In [86]:
import torch
import numpy as np
def _convert(inputs,labels):
    inputs = torch.from_numpy(inputs).type('torch.FloatTensor').cuda()
    labels = torch.from_numpy(labels).type('torch.LongTensor').cuda()
    return inputs,labels
def fit_wrapper(epoch_num,batch_size=32,epoch_shuffle=False,*args,**argws):
    def fit(model,data,optimizer,loss):
        data_ = {}
        #optimizer = optim.Adam(model.parameters(), lr=1e-2)
        train_metric = CategoricalMetric(4,-1)
        val_metric = CategoricalMetric(4,-1)
        for data_kind,item in data.items():
            data_[data_kind] = SeqGenerator(item['inputs'],item['answers'],
                                            batch_size,epoch_shuffle=epoch_shuffle,
                                            return_len=True,order='NCL',padding_value=-1,
                                            answer_one_hot=False)
        if 'validation' not in data.keys():
            data_['validation'] = None
        trian_data = data_['training']
        val_data = data_['validation']
        for epoch in range(epoch_num):  # loop over the dataset multiple times
            running_loss = 0.0
            for index, item in enumerate(trian_data):
                # get the inputs
                length, inputs, labels = item
                loss_ = train(model,inputs,labels,optimizer,loss,train_metric,length)
                # add loss
                running_loss += loss_
            if data_['validation'] is not None:
                val_loss = 0
                for item in data_['validation']:
                    length, inputs, labels = item
                    val_loss += validate(model,inputs,labels,loss,val_metric,length)
            #print(epoch + 1, running_loss,val_loss)
            print(epoch + 1, train_metric.accuracy,val_metric.accuracy)
            trian_data.on_epoch_end()
            val_data.on_epoch_end()
            train_metric.reset()
            val_metric.reset()
    return fit

In [34]:
def train(model,inputs,labels,optimizer,loss,metrics,length):
    inputs,labels = _convert(inputs,labels)
    optimizer.zero_grad()
    outputs = model(inputs,mask=length)
    outputs = outputs.contiguous()
    outputs = outputs.view(-1, outputs.size()[1])
    labels = labels.view(-1)
    metrics(F.softmax(outputs,dim=1).max(1)[1],labels)
    loss_ = loss(outputs, labels)
    loss_.backward()
    optimizer.step()
    
    return loss_.item()
def validate(model,inputs,labels,loss,metrics,length):
    inputs,labels = _convert(inputs,labels)
    outputs = model(inputs,length)
    outputs = outputs.contiguous()
    outputs = outputs.view(-1, outputs.size()[1])
    labels = labels.view(-1)
    metrics(F.softmax(outputs,dim=1).max(1)[1],labels)
    loss_ = loss(outputs, labels)
    return loss_.item()

In [177]:
from sequence_annotation.model.model_processor import SimpleModel,ModelCreator,IModelProcessor
import torch.nn as nn
import torch.nn.functional as F
class CNN_1D(nn.Module):
    def __init__(self,use_cuda=True,*args,**kwargs):
        super().__init__()
        self._kernel_size = kwargs['kernel_size']
        self.conv = nn.Conv1d(*args,**kwargs)
        if use_cuda:
            self.conv=self.conv.cuda()
        self.activation=nn.ReLU()
    def forward(self,x):
        x = F.pad(x, [0,self._kernel_size-1], 'constant', 0)
        x = self.conv(x)
        x = self.activation(x)
        return x
def run_rnn(rnn):
    def run(x,mask):
        x = torch.transpose(x, 1, 2)
        x = torch.nn.utils.rnn.pack_padded_sequence(x, mask, batch_first=True)
        x = rnn(x,mask)[0]
        x,len_ = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)
        x = torch.transpose(x, 1, 2)
        return x
    return run

class IndRNN(nn.Module):
    def __init__(self,input_size,hidden_size,use_cuda=True):
        super().__init__()
        self._hidden_size = hidden_size
        self._weight = nn.Conv1d(in_channels=input_size,kernel_size=1,
                                 out_channels=hidden_size)
        self._hidden = Variable(torch.zeros(1, hidden_size))
        if use_cuda:
            self._weight = self._weight.cuda()
            self._hidden = self._hidden.cuda()
        self.activation=nn.ReLU()
    def reset(self):
        self._hidden = Variable(torch.zeros(1, hidden_size))
    def forward(self,x,mask=None):
        #outputs = []
        num,channel,length = x.size()
        print(x.size())
        x = self._weight(x)
        print(x.size())
        x = torch.transpose(x, 0, 2)
        x = torch.transpose(x, 1, 2)
        outputs = torch.zeros([length, num,self._hidden_size])
        print(x.size())
        for i in range(x.size()[0]):
            self._hidden = self.activation(self._hidden + x[i])
            #print(i,self._hidden.size())
            #outputs.append()
        #x,len_ = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)
        #print(x.size())
        x = torch.transpose(x, 0, 2)
        x = torch.transpose(x, 1, 2)
        return x

class Model(nn.Module):
    def __init__(self,use_cuda=True):
        super().__init__()
        self.conv1 = CNN_1D(in_channels=4,kernel_size=32,out_channels=4,use_cuda=use_cuda)
        self.conv2 = CNN_1D(in_channels=4,kernel_size=112,out_channels=4,use_cuda=use_cuda)
        #self.rnn = RNN(input_size=4,hidden_size=4,batch_first=True,num_layers=1,use_cuda=use_cuda)
        self.rnn = IndRNN(input_size=4,hidden_size=40)
        #self.softmax = nn.log_Softmax(1)
    def forward(self, x, mask=None):
        num,channel,length = x.size()
        x = self.conv1(x)
        x = self.rnn(x,mask)
        x = self.conv2(x)
        #x = self.softmax(x)
        
        return x

In [178]:
from keras import optimizers
from keras.optimizers import Adam
from sequence_annotation.processor.metric.metric import BatchCount,TruePositive,SampleCount,Accuracy
from sequence_annotation.processor.data_processor import AnnSeqData,SimpleData
from sequence_annotation.pipeline.pipeline import Pipeline

In [179]:
criterion = nn.CrossEntropyLoss(reduction='mean',ignore_index=-1)
def optimizer(model):
    return optim.Adam(model.parameters(), lr=1e-2)

builder = SimpleModel(Model())
compiler = SimpleCompiler(optimizer,criterion)
data = AnnSeqData({'data':{'training':{'inputs':selected_fasta,
                                       'answers':selected_seqs}
                          },'ANN_TYPES':selected_seqs.ANN_TYPES},
                  discard_invalid_seq=True,validation_split=0.5,
                  do_validate=False,padding_value=None,
                  answer_one_hot=False
                 )
worker = TrainWorker(is_verbose_visible=True)
worker.wrapper = fit_wrapper(1000,256)
pipeline = Pipeline(builder,data,compiler,worker,
                    is_prompt_visible=True)



In [180]:
pipeline.execute()

Processing data...
Processing model..
Compiling model...
Processing worker...
Executing...
Start working(2018-12-15 13:25:42)
torch.Size([5, 4, 3346])
torch.Size([5, 40, 3346])
torch.Size([3346, 5, 40])
0 torch.Size([5, 40])
1 torch.Size([5, 40])
2 torch.Size([5, 40])
3 torch.Size([5, 40])
4 torch.Size([5, 40])
5 torch.Size([5, 40])
6 torch.Size([5, 40])
7 torch.Size([5, 40])
8 torch.Size([5, 40])
9 torch.Size([5, 40])
10 torch.Size([5, 40])
11 torch.Size([5, 40])
12 torch.Size([5, 40])
13 torch.Size([5, 40])
14 torch.Size([5, 40])
15 torch.Size([5, 40])
16 torch.Size([5, 40])
17 torch.Size([5, 40])
18 torch.Size([5, 40])
19 torch.Size([5, 40])
20 torch.Size([5, 40])
21 torch.Size([5, 40])
22 torch.Size([5, 40])
23 torch.Size([5, 40])
24 torch.Size([5, 40])
25 torch.Size([5, 40])
26 torch.Size([5, 40])
27 torch.Size([5, 40])
28 torch.Size([5, 40])
29 torch.Size([5, 40])
30 torch.Size([5, 40])
31 torch.Size([5, 40])
32 torch.Size([5, 40])
33 torch.Size([5, 40])
34 torch.Size([5, 40])
35

513 torch.Size([5, 40])
514 torch.Size([5, 40])
515 torch.Size([5, 40])
516 torch.Size([5, 40])
517 torch.Size([5, 40])
518 torch.Size([5, 40])
519 torch.Size([5, 40])
520 torch.Size([5, 40])
521 torch.Size([5, 40])
522 torch.Size([5, 40])
523 torch.Size([5, 40])
524 torch.Size([5, 40])
525 torch.Size([5, 40])
526 torch.Size([5, 40])
527 torch.Size([5, 40])
528 torch.Size([5, 40])
529 torch.Size([5, 40])
530 torch.Size([5, 40])
531 torch.Size([5, 40])
532 torch.Size([5, 40])
533 torch.Size([5, 40])
534 torch.Size([5, 40])
535 torch.Size([5, 40])
536 torch.Size([5, 40])
537 torch.Size([5, 40])
538 torch.Size([5, 40])
539 torch.Size([5, 40])
540 torch.Size([5, 40])
541 torch.Size([5, 40])
542 torch.Size([5, 40])
543 torch.Size([5, 40])
544 torch.Size([5, 40])
545 torch.Size([5, 40])
546 torch.Size([5, 40])
547 torch.Size([5, 40])
548 torch.Size([5, 40])
549 torch.Size([5, 40])
550 torch.Size([5, 40])
551 torch.Size([5, 40])
552 torch.Size([5, 40])
553 torch.Size([5, 40])
554 torch.Size([

1175 torch.Size([5, 40])
1176 torch.Size([5, 40])
1177 torch.Size([5, 40])
1178 torch.Size([5, 40])
1179 torch.Size([5, 40])
1180 torch.Size([5, 40])
1181 torch.Size([5, 40])
1182 torch.Size([5, 40])
1183 torch.Size([5, 40])
1184 torch.Size([5, 40])
1185 torch.Size([5, 40])
1186 torch.Size([5, 40])
1187 torch.Size([5, 40])
1188 torch.Size([5, 40])
1189 torch.Size([5, 40])
1190 torch.Size([5, 40])
1191 torch.Size([5, 40])
1192 torch.Size([5, 40])
1193 torch.Size([5, 40])
1194 torch.Size([5, 40])
1195 torch.Size([5, 40])
1196 torch.Size([5, 40])
1197 torch.Size([5, 40])
1198 torch.Size([5, 40])
1199 torch.Size([5, 40])
1200 torch.Size([5, 40])
1201 torch.Size([5, 40])
1202 torch.Size([5, 40])
1203 torch.Size([5, 40])
1204 torch.Size([5, 40])
1205 torch.Size([5, 40])
1206 torch.Size([5, 40])
1207 torch.Size([5, 40])
1208 torch.Size([5, 40])
1209 torch.Size([5, 40])
1210 torch.Size([5, 40])
1211 torch.Size([5, 40])
1212 torch.Size([5, 40])
1213 torch.Size([5, 40])
1214 torch.Size([5, 40])


1918 torch.Size([5, 40])
1919 torch.Size([5, 40])
1920 torch.Size([5, 40])
1921 torch.Size([5, 40])
1922 torch.Size([5, 40])
1923 torch.Size([5, 40])
1924 torch.Size([5, 40])
1925 torch.Size([5, 40])
1926 torch.Size([5, 40])
1927 torch.Size([5, 40])
1928 torch.Size([5, 40])
1929 torch.Size([5, 40])
1930 torch.Size([5, 40])
1931 torch.Size([5, 40])
1932 torch.Size([5, 40])
1933 torch.Size([5, 40])
1934 torch.Size([5, 40])
1935 torch.Size([5, 40])
1936 torch.Size([5, 40])
1937 torch.Size([5, 40])
1938 torch.Size([5, 40])
1939 torch.Size([5, 40])
1940 torch.Size([5, 40])
1941 torch.Size([5, 40])
1942 torch.Size([5, 40])
1943 torch.Size([5, 40])
1944 torch.Size([5, 40])
1945 torch.Size([5, 40])
1946 torch.Size([5, 40])
1947 torch.Size([5, 40])
1948 torch.Size([5, 40])
1949 torch.Size([5, 40])
1950 torch.Size([5, 40])
1951 torch.Size([5, 40])
1952 torch.Size([5, 40])
1953 torch.Size([5, 40])
1954 torch.Size([5, 40])
1955 torch.Size([5, 40])
1956 torch.Size([5, 40])
1957 torch.Size([5, 40])


KeyboardInterrupt: 

In [ ]:
i=torch.from_numpy(np.array([[1,0,0],[0,-230,1000]])).type('torch.FloatTensor').cuda()
F.softmax(i,dim=1)

In [11]:

o=torch.from_numpy(np.array([1,1,2])).type('torch.LongTensor').cuda()
criterion = nn.NLLLoss(reduction='none')
criterion(log(i),o)

NameError: name 'log' is not defined

In [ ]:
F.log_softmax